# Densest subgraph

## Basic Graph Class

We already provide the basic implementation of a graph class.

Iterating over nodes can be done by using `for u in G.nodesToEdges` and iterating over the edges incident upon node `u` can be done using `for v in G.nodesToEdges[u]`. To check whether an edge $(u,v)$ exists use `G.edgeExists(u,v)`.

Graphs can be read from files using the `readFromFile`-procedure. Using the optional argument `verticesToIgnore` which expects a `set()` as input, you may exclude some nodes from the graph, i.e., if `verticesToIgnore` is some set $S$ then it will load the graph $G=(V\setminus S,E[V\setminus S])$. Note that you might have to delete header-rows from some data files to make the procedure work.

In [16]:
class Graph:
    def __init__(self):
        self.numNodes = 0
        self.numEdges = 0
        self.edges = set()
        self.nodesToEdges = {}

    def addEdge(self, u, v):
        if u == v or v in self.nodesToEdges and u in self.nodesToEdges[v]:
            return
        self.numEdges += 1
        self.addNeighbor(u, v)
        self.addNeighbor(v, u)
        self.edges.add((u, v))

    def removeEdge(self, u, v):
        self.numEdges -= 1
        self.nodesToEdges[u].remove(v)
        self.nodesToEdges[v].remove(u)
        if (u, v) in self.edges:
            self.edges.remove((u, v))
        else:
            self.edges.remove((v, u))

    def addNeighbor(self, u, v):
        if u not in self.nodesToEdges:
            self.numNodes += 1
            self.nodesToEdges[u] = set()
        self.nodesToEdges[u].add(v)

    def edgeExists(self, u, v):
        return ((u, v) in self.edges or (v, u) in self.edges)

    def degree(self, u):
        return len(self.nodesToEdges[u])

    def readFromFile(self, filePath, separator=',', verticesToIgnore=set()):
        with open(filePath, 'r') as f:
            next(f)
            for line in f:
                split = line.split(separator)
                u = int(split[0])
                v = int(split[1])
                if u in verticesToIgnore or v in verticesToIgnore:
                    continue
                self.addEdge(u, v)
        print(f'Finished reading graph with {self.numNodes} nodes and {self.numEdges} edges.')

## Basic Linked List Data Structure

First, implement your own linked list data structure ```LinkedList``` in which each element is from the class ```LinkedListElement```.

In [34]:
class LinkedListElement:
    def __init__(self,id,prevElement=None,nextElement=None):
        self.id = id
        self.next = nextElement
        self.prev = prevElement

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self.size = 0

    def appendElement(self, element: int):
        new_node = LinkedListElement(element)
        if not self.tail:
            self.head = new_node
            self.tail = new_node
        else:
            new_node.prev = self.tail
            self.tail.next = new_node
            self.tail = new_node
        self.size += 1
        return new_node

    def removeElement(self, element: int):

        if not self.head:
            return
        
        if element == self.head:
            self.head = element.next
            if self.head:
                self.head.prev = None
            else:
                self.tail = None
        
        elif element == self.tail:
            self.tail = element.prev
            if self.tail:
                self.tail.next = None
            else:
                self.head = None
        
        else:
            if element.prev:
                element.prev.next = element.next
            if element.next:
                element.next.prev = element.prev
        
        # Decrement size
        self.size -= 1

    def pop(self):
        temp = self.tail
        if not temp:
            return
        data = temp.id
        self.tail = temp.prev
        if not self.tail:
            self.head = None
        else:
            self.tail.next = None
        self.size -= 1
        return data

    def __len__(self):
        return self.size

## Implementation of the Greedy Peeling Algorithm

Next, implement the greedy peeling algorithm. The function should be callable using ```densestSubgraphGreedyPeeling(G)``` where ```G``` is a graph using the graph class from above. Besides G, the function may accept more optional arguments.


In [57]:
def densestSubgraphGreedyPeeling(G: Graph):
    n = G.numNodes
    edges = G.edges
    nodesToEdges = G.nodesToEdges
    d = [0] * G.numNodes *2
    
    # Count degrees
    for u, v in edges:
        d[u] += 1
        d[v] += 1
    
    # Initialize linked lists for each degree
    L : list[LinkedList] = [LinkedList() for _ in range(max(nodesToEdges.keys()))]
    vertex_pointers = {}  # Map vertex to its node in L
    remaining_vertices : set[int] = set(nodesToEdges.keys())
    # Add vertices to appropriate degree lists
    for v in remaining_vertices:
        node = L[d[v]].appendElement(v)
        vertex_pointers[v] = node


    remaining_edges = len(edges)
    max_density = remaining_edges / len(remaining_vertices)
    best_iteration = 0
    ks = 0  # Minimum degree pointer


    
    # Create a copy to track removed vertices for the result
    removed_vertices = []
    ks : int = 0
    # Main loop
    for i in list(remaining_vertices):


        density = remaining_edges / len(remaining_vertices)
        if density > max_density:
            max_density = density
            best_iteration = len(removed_vertices) + 1

        while len(L[ks]) == 0:
            ks += 1
        

        v = L[ks].pop()
        removed_vertices.append(v)
        remaining_vertices.remove(v)
        
        # Remove edges connected to v
        for u in nodesToEdges[v]:
            if u == v:
                remaining_edges -= 1

            if u in remaining_vertices:
                remaining_edges -= 1

                d[u] -= 1
                if d[u] < ks:
                    ks = d[u]

                L[d[u]+1].removeElement(vertex_pointers[u])

                node = L[d[u]].appendElement(u)
                vertex_pointers[u] = node
        


    
    # Return the densest subgraph
    densest_subgraph = list(nodesToEdges.keys())
    for i in range(best_iteration):

        densest_subgraph.remove(removed_vertices[i])

    
    return max_density, densest_subgraph

## Experiments for Densest Subgraph

Now, the experiments for the two datasets follow.

In [58]:
from copy import deepcopy
from collections import defaultdict

def extract_disjoint_dense_subgraphs(G: Graph, num_subgraphs=5):

    current_G: Graph = deepcopy(G)
    current_edges = G.edges.copy()
    

    
    # Store results: (density, subgraph) for each round
    results = []
    
    for i in range(num_subgraphs):
        # Skip if graph is empty
        if not current_G:
            print(f"No more vertices available after extracting {i} subgraphs")
            break
            

        density, subgraph = densestSubgraphGreedyPeeling(current_G)
        print(f"Density: {density}, Size: {len(subgraph)}")
        # Add to results
        results.append((density, subgraph))
        

        
        # Create new graph for next iteration by removing found subgraph vertices
        new_G = Graph()
        new_edges = []
        
        for v, edges in current_G.nodesToEdges.items():
            if v not in subgraph:
                for u in edges:
                    new_G.addEdge(v, u)
                #new_G[v] = [u for u in current_G[v] if u not in subgraph]

        # for u, v in current_edges:
        #     if u not in subgraph and v not in subgraph:
        #         new_edges.append((u, v))
        
        current_G = new_G
        
        print(f"Found subgraph with density {density} and {len(subgraph)} vertices")
    
    return results

### Experiments for the OpenFlights Dataset

In [59]:
g= Graph()
g.readFromFile('flight/edges.csv')
density, subgraph = densestSubgraphGreedyPeeling(g)

Finished reading graph with 3214 nodes and 18858 edges.


In [60]:
r = extract_disjoint_dense_subgraphs(g, 5)

Density: 25.372881355932204, Size: 176
Found subgraph with density 25.372881355932204 and 176 vertices
Density: 16.84375, Size: 63
Found subgraph with density 16.84375 and 63 vertices
Density: 11.39344262295082, Size: 60
Found subgraph with density 11.39344262295082 and 60 vertices
Density: 10.373134328358208, Size: 334
Found subgraph with density 10.373134328358208 and 334 vertices
Density: 6.8545454545454545, Size: 109
Found subgraph with density 6.8545454545454545 and 109 vertices


### Experiments for the Facebook Dataset

In [61]:
g= Graph()
g.readFromFile('facebook_large/musae_facebook_edges.csv')
density, subgraph = densestSubgraphGreedyPeeling(g)

Finished reading graph with 22470 nodes and 170823 edges.


In [62]:
r = extract_disjoint_dense_subgraphs(g, 5)

Density: 35.0, Size: 323
Found subgraph with density 35.0 and 323 vertices
Density: 25.359550561797754, Size: 88
Found subgraph with density 25.359550561797754 and 88 vertices
Density: 22.218934911242602, Size: 844
Found subgraph with density 22.218934911242602 and 844 vertices
Density: 17.345454545454544, Size: 54
Found subgraph with density 17.345454545454544 and 54 vertices
Density: 15.466453674121405, Size: 312
Found subgraph with density 15.466453674121405 and 312 vertices


## Experiments for Optimal Quasi Clique

Now re-run the experiments from above with the new objective function and compare the results to the previous outputs.

### Experiments for the OpenFlights Dataset

### Experiments for the Facebook Dataset